In [267]:
import re
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio

# Config
pio.templates.default = "plotly_dark"

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)
pd.set_option("plotting.backend", "plotly")

# TODO: Backtesting

# Data

In [268]:

data = pd.read_csv("data.csv")
data_filtered = data.drop(columns=data.columns[3:5])

courses = ["ENGLISH100", "ARB100", "MATH101", "CHEM101", "STAT101", "TECH101", "ENTREPRENEUR101", "FAJAB101", "NAHAJ101", "ENGLISH110",
            "SALAM107", "PHYS104", "MATH106", "CSC111", "MATH151", "SALAM108", "CSC113", "CSC220", "MATH244",
            "CSC212", "CSC215", "MATH281", "CSC304", "CSC380", "CSC227", "CSC311", "CSC339", "CSC343", "CSC361",
            "CSC329", "CSC340", "CSC453", "CSC496", "PHYS210", "PHYS103", "CHEM103", "CSC443", "CSC462", "CSC476", "CSC478", "CSC484", "CSC489"]
prefix_to_category = {"ENGLISH":"Humanities", "NAHAJ": "Humanities", "ARB": "Humanities", "ENTREPRENEUR": "Humanities", "FAJAB": "Humanities", "TECH": "Humanities",
                    "PHYS": "Physics", "CHEM": "Chemistry", "SALAM": "Islamics", "MATH": "Mathematics", "STAT": "Mathematics", "CSC": "Computer science"}
criteria = ["Application", "Relevance", "Insight", "Understanding", "Ease"]
ratings = list(range(1, 6))

In [269]:
categories = pd.Series(re.findall("[a-zA-Z]+", "\n".join(courses))).apply(prefix_to_category.get)
assert sum(categories.isna()) == 0, "prefix_to_category must cover each course prefix"

# Wrangling

In [270]:
def df_by_regex(regex):
    """THIS USES GLOBALY DEFINED DF ALARM ALARM"""
    return data_filtered.filter(regex=regex, axis=1)

def get_subject_df(subject):
    """THIS USES GLOBALY DEFINED DF ALARM ALARM"""
    return df_by_regex(f"{subject}.*\[.*\].*")

In [271]:
index = list((course, criterion) for criterion in criteria for course in courses)
sorted_index = sorted(index, key=lambda x: courses.index(x[0]))
df = get_subject_df("").T.set_index(pd.Index(sorted_index))

## Ranking

### What course is [max, min, contriversial] based on [criteria, weights]

In [334]:
# course_to_application_score_mean.reset_index()["index"] = categories
course_to_application_score_mean.index = categories
c = course_to_application_score_mean.reset_index().groupby("index").mean()
course_to_application_mean = c

In [335]:
c

,0
index,
Chemistry,1.500000
Computer science,3.464448
Humanities,2.990646
Islamics,2.328571
Mathematics,4.011905
Physics,1.547619


In [333]:
course_to_application_score_mean.reset_index()

,index,0
0,Humanities,4.000000
1,Humanities,2.625000
2,Mathematics,3.750000
3,Chemistry,2.000000
4,Mathematics,4.571429
5,Humanities,3.142857
6,Humanities,2.285714
7,Humanities,2.166667
8,Humanities,2.857143
9,Humanities,3.857143


In [301]:
course_to_application_score_mean = df.loc[[(subject, "Application") for subject in courses]].T.mean().droplevel(1)
course_to_application_score_mean.reset_index()
fig = px.scatter(course_to_application_score_mean, color=categories, symbol=categories)
fig.update_layout(
    title="Course Vs. Application score",
    xaxis_title="Course",
    yaxis_title="Application score (1-5)",
    legend_title="",
)

## Meta-stats

In [ ]:
# a bit of a primitive way to count every filled in ranking (some rankings are NaN). Maybe it could be improved?
i = 0
course_to_num_ranking = {}
while i < len(df):
    course_to_num_ranking.update({df.iloc[i].name[0]:len(df.iloc[i].dropna())})
    i += 5
course_to_num_ranking_series = pd.Series(course_to_num_ranking)

In [ ]:
px.scatter(course_to_num_ranking_series, color=categories, symbol=categories, hover_name=course_to_num_ranking_series.index)

In [ ]:
rating_counts_per_criteria = []
for criterion in criteria:
    rating_counts_per_criteria.append(df_by_regex(f".*{criterion}.*").apply(pd.value_counts).sum(axis=1))
rating_counts_df = pd.DataFrame(rating_counts_per_criteria).T
rating_counts_df.columns = criteria

In [ ]:
rating_counts_df

,Application,Relevance,Insight,Understanding,Ease
1.0,40.0,35.0,51.0,36.0,28.0
2.0,30.0,31.0,30.0,26.0,28.0
3.0,40.0,42.0,40.0,38.0,45.0
4.0,26.0,36.0,39.0,44.0,45.0
5.0,86.0,78.0,62.0,78.0,76.0


In [ ]:
rating_counts_raw_criteria = []
for criterion in criteria:
    rating_counts_raw_criteria.append(np.concatenate(df_by_regex(f".*{criterion}.*").values))
rating_counts_raw_df = pd.DataFrame(rating_counts_raw_criteria).T
rating_counts_raw_df.columns = criteria

In [ ]:
px.violin(rating_counts_raw_df)

In [ ]:
px.histogram(df_by_regex(".*Application.*"))

In [ ]:
px.histogram(np.concatenate(df_by_regex(f".*Relevance.*").values))

In [ ]:
px.histogram(df["MATH101 [Application]"])